<a href="https://colab.research.google.com/github/e18679/ML0930/blob/master/sentiment_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",   
    extract=True,
)

84131840/84125825 [==============================] - 7s 0us/step


In [2]:
import glob
# glob.glob("/root/.keras/datasets/aclImdb/train/pos/*")
import os
import pandas as pd
def read(path):
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()
    return content

def get_data(t):
    dn = os.path.dirname(dataset)
    pattern = os.path.join(dn, "aclImdb", t, "pos", "*.txt")
    pos = glob.glob(pattern)
    pattern = os.path.join(dn, "aclImdb", t, "neg", "*.txt")
    neg = glob.glob(pattern)
    sentiments = [1] * len(pos) + [0] * len(neg)
    contents = map(read, pos + neg)
    df = pd.DataFrame({
        "contents":contents,
        "sentiment":sentiments
    })
    return df

train_df = get_data("train")
test_df = get_data("test")

In [3]:
TOK = 3000
LEN = 512
EM = 128

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
# fit_on_texts: fit_transform 的 fit
tok.fit_on_texts(train_df["contents"])
# seq: 把所有單字換成數字的序列
x_train_seq = tok.texts_to_sequences(train_df["contents"])
x_test_seq = tok.texts_to_sequences(test_df["contents"])

In [5]:
# tok.word_index
tok.index_word[2]

'and'

In [6]:
# pd.DataFrame(train_df_seq)
# truncating: pre 12345 -> 45 post 12345 -> 12
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, LEN)
x_test_pad = pad_sequences(x_test_seq, LEN)
pd.DataFrame(x_test_pad)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,8,3,1161,12,1,17,6,21,1867,14,45,12,68,3,2178,15,49,19,280,54,39,1029,8,11,28,40,3,706,487,178,5,121,86,11,62,627,277,22,377,146
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,13,31,666,2041,1532,1,1437,606,37,3,247,8,58,702,9,13,37,316,32,776,877,9,90,3,524,133,12,73,430,276,54,28,77,166,11,730,18,9,1674,69
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,728,196,1,2,1,1393,1,106,13,1,29,55,115,265,1016,431,53,5,12,55,2,125,71,256,234,1558,6,305,7,7,92,171,48,78,10,121,143,61,3,504
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4,792,23,134,22,200,64,86,9,492,8,1,1844,162,10,101,11,6,28,4,1,115,245,284,2,10,481,383,11,5,29,346,34,37,369,7,7,15,58,1911
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,97,1,254,248,6,32,1,995,1489,1847,23,2437,18,1,19,14,3,223,6,21,10,188,101,4,157,422,12,67,1556,1,191,8,3,278,4,578,476,20,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,10,822,12,230,2315,50,71,3,644,1755,8,1,624,485,307,35,5,10,479,10,25,5,137,16,1,30,219,9,405,1722,99,3,321,30,1,1710,1064,1214,1688,2335
24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,169,2752,2,26,57,8,108,47,23,197,131,104,99,1,290,1467,109,12,109,47,1098,364,2,149,20,1464,2,5,984,3,582,134,995,180,72,190,15,2484,8,110
24997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,12,1,84,81,33,23,266,5,27,162,119,11,91,217,14,45,33,801,5,898,8,14,108,2869,14,611,47,23,108,49,1544,105,2,11,6,21,28,4,95
24998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,299,6,48,163,11,17,287,2739,7,7,1,315,941,1468,3,1234,646,80,1,174,2,1442,1919,4,1419,7,7,511,344,1,4,58,2037,177,5,11,52,558,4,55


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.layers import SimpleRNN

layers = [
  # 1~3000(token) + 0(padding)
  # params: 3001 * 128
  Embedding(TOK+1, EM, mask_zero=True, input_length=LEN),
  SimpleRNN(64),
  Dense(2, activation="softmax")  
]
model = Sequential(layers)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 512, 128)          384128    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 64)                12352     
_________________________________________________________________
dense (Dense)                (None, 2)                 130       
Total params: 396,610
Trainable params: 396,610
Non-trainable params: 0
_________________________________________________________________


In [8]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer=Adam(),
       metrics=["accuracy"])

In [9]:
y_train = train_df["sentiment"]
y_test = test_df["sentiment"]

In [10]:
# epoch: 整份資料要看幾遍
# 60000筆, 2epoch, 200batch -> 60000 * 2 / 200次調整
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
    ModelCheckpoint("model.h5", save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True)
]
model.fit(x_train_pad, 
     y_train,
     batch_size=100,
     epochs=50,
     validation_split=0.1,
     callbacks=callbacks,
     verbose=2)

Epoch 1/50
225/225 - 78s - loss: 0.5623 - accuracy: 0.6944 - val_loss: 0.3168 - val_accuracy: 0.8816
Epoch 2/50
225/225 - 76s - loss: 0.3873 - accuracy: 0.8283 - val_loss: 0.5909 - val_accuracy: 0.7068
Epoch 3/50
225/225 - 77s - loss: 0.2816 - accuracy: 0.8835 - val_loss: 0.5367 - val_accuracy: 0.8116
Epoch 4/50
225/225 - 77s - loss: 0.2101 - accuracy: 0.9128 - val_loss: 0.9056 - val_accuracy: 0.5652
Epoch 5/50
225/225 - 77s - loss: 0.1314 - accuracy: 0.9517 - val_loss: 0.7441 - val_accuracy: 0.7684
Epoch 6/50
225/225 - 78s - loss: 0.0543 - accuracy: 0.9826 - val_loss: 1.0875 - val_accuracy: 0.7148


In [11]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 44s 57ms/step - loss: 0.4095 - accuracy: 0.8224


[0.4094679355621338, 0.8224400281906128]

In [ ]:
layers = [
  Embedding(TOK+1, EM, mask_zero=True),
  GlobalAveragePooling1D()
]
w = model.layers[0].get_weights()
infer = Sequential(layers)
infer.layers[0].set_weights(w)
infer.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         384128    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 128)               0         
Total params: 384,128
Trainable params: 384,128
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# w
# infer.layers[0].get_weights()
# 25000, 512
# [512list, 512list]
target = "the"
# [1list]
pre = infer.predict([[tok.word_index[target]]])
pre[0]

array([ 1.14329625e-02,  2.59512682e-02, -6.09875750e-03, -1.96145307e-02,
        1.59987751e-02, -7.09439162e-03, -2.12033987e-02, -2.55701318e-02,
       -3.04902159e-02, -6.06370680e-02,  1.20779164e-02,  1.45602738e-02,
       -2.27468014e-02,  1.30747976e-02, -5.48306480e-02, -5.56809045e-02,
        1.32313753e-02,  4.55633141e-02,  1.09513244e-02, -3.15188766e-02,
        6.56417906e-02,  1.42393652e-02,  5.55761009e-02,  3.44350487e-02,
        5.89647144e-02, -5.24996705e-02,  1.45676185e-03,  1.61569491e-02,
       -4.88803685e-02, -2.06186976e-02, -7.76312605e-04, -1.40861543e-02,
       -1.94320846e-02, -2.86915433e-02, -5.90931065e-03, -1.54673168e-02,
       -9.38439462e-03,  1.59146357e-02,  4.91407178e-02, -1.21285049e-02,
       -4.47066221e-03,  2.70790700e-02,  3.83190885e-02,  5.12493961e-02,
        1.13119213e-02,  4.06878144e-02, -3.95480841e-02,  2.78827548e-02,
        1.76851694e-02,  5.73802777e-02, -5.98486178e-02,  1.36457272e-02,
        9.33653954e-03,  

In [ ]:
f = open("vec.txt", "w", encoding="utf-8")
f.write("{} {}\n".format(TOK, EM))
for i in range(1, TOK+1):
    # [1list]
    pre = infer.predict([[i]])[0]
    v = " ".join(map(str, pre))
    f.write("{} {}\n".format(tok.index_word[i], v))
f.close()

In [ ]:
from gensim.models import KeyedVectors
w2v = KeyedVectors.load_word2vec_format("vec.txt")

In [ ]:
w2v.wv.vocab

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


{'the': <gensim.models.keyedvectors.Vocab at 0x7efd6dfb2c88>,
 'and': <gensim.models.keyedvectors.Vocab at 0x7efd6dfb28d0>,
 'a': <gensim.models.keyedvectors.Vocab at 0x7efd6dfb2e48>,
 'of': <gensim.models.keyedvectors.Vocab at 0x7efd6dfb2dd8>,
 'to': <gensim.models.keyedvectors.Vocab at 0x7efd365ba080>,
 'is': <gensim.models.keyedvectors.Vocab at 0x7efd365bae48>,
 'br': <gensim.models.keyedvectors.Vocab at 0x7efd365ba160>,
 'in': <gensim.models.keyedvectors.Vocab at 0x7efd27613b38>,
 'it': <gensim.models.keyedvectors.Vocab at 0x7efd268894e0>,
 'i': <gensim.models.keyedvectors.Vocab at 0x7efd268897f0>,
 'this': <gensim.models.keyedvectors.Vocab at 0x7efd26444908>,
 'that': <gensim.models.keyedvectors.Vocab at 0x7efd263ed438>,
 'was': <gensim.models.keyedvectors.Vocab at 0x7efd263ed470>,
 'as': <gensim.models.keyedvectors.Vocab at 0x7efd263ed4a8>,
 'for': <gensim.models.keyedvectors.Vocab at 0x7efd263ed4e0>,
 'with': <gensim.models.keyedvectors.Vocab at 0x7efd263ed518>,
 'movie': <gensi

In [ ]:
# 詞彙列表: w2v.wv.vocab
w2v.most_similar("joke")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('worst', 0.9944390058517456),
 ('pointless', 0.9943830966949463),
 ('unfortunately', 0.9940879940986633),
 ('waste', 0.9940686821937561),
 ('obnoxious', 0.9940173625946045),
 ('poor', 0.9939393997192383),
 ('annoying', 0.9938967227935791),
 ('laughable', 0.9937937259674072),
 ('ridiculous', 0.9937877655029297),
 ('unless', 0.9937089085578918)]